In [79]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [80]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNWUwMGIxYTYtMWZmOS00OTJhLWE3ODMtZmZiNjU1NGE4MTcyIiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJyZWNpcGllbnRLZXlzIjogWyJBWVp4ZVhKaEM2WHV2TFJCTDZaMjllSHpDb1FMS2tSblh3OVpwRlVMNHBjeiJdLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly9hZ2VudDEubG9jYWxob3N0In0=


In [81]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': '5e00b1a6-1ff9-492a-a783-ffb6554a8172',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Main (admin)',
 'recipientKeys': ['AYZxeXJhC6XuvLRBL6Z29eHzCoQLKkRnXw9ZpFUL4pcz'],
 'serviceEndpoint': 'http://agent1.localhost'}


In [82]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 LH5Lk3yECKjR2NW6bph5LrdcDVaTD683YHAUmzn1j3rnXfA8nJrw1LoE9RwV7nMrFcEgaanLw6DoZ5ESthjAb5S

Public key: 
 6uZ7fyYsAmvwszTQZ4rxyKo2NG9NN77SfQeSgeY3SNgY


**Sending a connection request to acapy**

In [83]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [84]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")

In [85]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [86]:
   
connectionRequestResponseUnpacked = \
    util.unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "05ac719c-9902-40ee-9fd0-fd82b8d27f64", "~thread": {"thid": '
 '"c2371658-4cc3-42ce-b7ce-a050c86dd301"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"6A9bOjrYjmfXJ8gPo4egLPe_zBhEJvMmavwkfE7zT3-qTTlz4IW5SpWlHrXF5WtJUygbWpLqxsdt79pSN9iUDg==", '
 '"sig_data": '
 '"AAAAAF7FIYl7IkRJRCI6ICJRTmZiTlZ0WnYxUVNnU0pVaWc3WHZGIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpRTmZiTlZ0WnYxUVNnU0pVaWc3WHZGIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6UU5mYk5WdFp2MVFTZ1NKVWlnN1h2RiMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OlFOZmJOVnRadjFRU2dTSlVpZzdYdkYiLCAicHVibGljS2V5QmFzZTU4IjogIkRqdEZwWEw0SDJNZnIzR3BQYWpHUWJVN1JUWVNmTmNMY3hWM0tvcTlNQ1hVIn1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [87]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
#  (this is a hack)replacing artifacts that sometimes happen
sig_data_raw = sig_data_raw.replace("-", "1")
sig_data_raw = sig_data_raw.replace("_", "1")

sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'QNfbNVtZv1QSgSJUig7XvF',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:QNfbNVtZv1QSgSJUig7XvF#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:QNfbNVtZv1QSgSJUig7XvF',
            'publicKey': [{'controller': 'did:sov:QNfbNVtZv1QSgSJUig7XvF',
                           'id': 'did:sov:QNfbNVtZv1QSgSJUig7XvF#1',
                           'publicKeyBase58': 'DjtFpXL4H2Mfr3GpPajGQbU7RTYSfNcLcxV3Koq9MCXU',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:QNfbNVtZv1QSgSJUig7XvF;indy',
                         'priority': 0,
                         'recipientKeys': ['DjtFpXL4H2Mfr3GpPajGQbU7RTYSfNcLcxV3Koq9MCXU'],
                         'serviceEndpoint': 'http://agent1.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [88]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:QNfbNVtZv1QSgSJUig7XvF#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:QNfbNVtZv1QSgSJUig7XvF',
             'publicKey': [{'controller': 'did:sov:QNfbNVtZv1QSgSJUig7XvF',
                            'id': 'did:sov:QNfbNVtZv1QSgSJUig7XvF#1',
                            'publicKeyBase58': 'DjtFpXL4H2Mfr3GpPajGQbU7RTYSfNcLcxV3Koq9MCXU',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:QNfbNVtZv1QSgSJUig7XvF;indy',
                          'priority': 0,
                          'recipientKeys': ['DjtFpXL4H2Mfr3GpPajGQbU7RTYSfNcLcxV3Koq9MCXU'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'\x10\x9f\xb6\xe3\x9a\xf2\xa5\x99\x82J\xd4Tl\x0f\x89|j\x8

### FEATURE DISCOVERY

In [89]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
    query="*"
)
message = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=message)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '4b5db1d2-aac7-404f-ad33-5121021cd16f',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/trust_ping/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-credential-definitions/0.1'},
               {'pid': 'https://didcomm.org/notification/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-taa/0.1'},
               {'pid': 'https://didcomm.org/issue-credential/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-issuance/0.1'},
               {'pid': 'https://didcomm.org/introduction-service/0.1'},
               {'pid': 'https://didcomm.org/routing/1.0'},
               {'pid':

In [92]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records/add",
    payload="{amazing: 'payload'}"
)

encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '7b26dc60-5549-4846-8602-ab5b076681bc',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records/add',
 'payload': "{amazing: 'payload'}",
 'record_id': '63f405be7faf497cb687e44e8cb7d9a1',
 '~thread': {'thid': '18fd7e5f-fa3b-4d49-8e59-11df491de88b'}}


In [96]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records/get",
    record_id="2a272d933be14d9e9a92589395ed75db"
)

encodedMessage = util.packMessage(message, connections[0])

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'cdbf3323-791b-455c-94fb-fbe21a21d09c',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records/add',
 'payload': "{amazing: 'payload'}",
 'record_id': '2a272d933be14d9e9a92589395ed75db',
 '~thread': {'thid': 'f0ccc52c-344a-46c4-9ca3-422b418a7e01'}}
